In [1]:
import praw
import pandas as pd
from tqdm import tqdm
import time

In [2]:
reddit = praw.Reddit(
    client_id="y25BJERt0d-YtQKuTk7gUQ",
    client_secret="fBb7prfE4DcIj2Cv4l7Dw73ziLZFyQ",
    user_agent="sl-nlp-project:v1 (by /u/SpareUse7114)", 
)
reddit.read_only

True

In [3]:
subreddit = reddit.subreddit("srilanka")
subreddit

Subreddit(display_name='srilanka')

In [4]:
keywords = [
    "Sri Lanka","Colombo","Ampara","Anuradhapura","Badulla","Batticaloa","Galle","Gampaha","Hambantota","Jaffna","Kalutara","Kandy","Kegalle","Kilinochchi",
    "Kurunegala","Mannar","Matale","Matara","Monaragala","Mullaitivu","Nuwara Eliya","Polonnaruwa","Puttalam","Ratnapura","Trinco","Vavuniya","Phone","Public",
    "Negombo","Moratuwa","Kotte","Kalmunai","Beruwala","Chilaw","Panadura","Wattala","Katunayake","Haputale","Hatton","Bandarawela","Bentota","Laptop","Private",
    "Hikkaduwa","Weligama","Tangalle","Pedro","Kalpitiya","Pettah","Island","Dehiwala","Lavinia","Maharagama","Kelaniya","Cuisine","Game","Hero","AI","Policy",
    "Ceylon","Sigiriya","Peak","Horton Plains","Ella","Dambulla","Mirissa","Arugam Bay","Poya","Kataragama","Temple","Kovil","Dress","Food","Tournament","Progress"
    "Yala","Wilpattu","Udawalawe","Knuckles","Sinharaja","Nine Arches Bridge","Fort","Ravana","Pidurangala","Lotus Tower","Independence","Home","Tool","Problem",
    "Victoria","Gregory","Beira","Beach","Unawatuna","Nilaveli","Pasikuda","Church","Mosque","Sinhala","Tamil","Islam","Catholic","Rupee","LKR","Port","Solution",
    "Airlines","Tea","Coconut","Rice","Spices","Ayurveda","Parliament","President","Prime Minister","SLPP","UNP","SJB","JVP","NPP","EPDP","TNA","Bank","Support",
    "CSE","Colombo Stock Exchange","Police","Army","Navy","Air","Force","Perahera","Thai Pongal","Deepavali","Ramadan","Eid","Christmas","Bodhi","Tree","Concern",
    "Dagoba","Dewalaya","Tourism","Economy","Export","Import","Garment","Rubber","Cinnamon","Cardamom","Pepper","Cinnamon","Sapphire","Gem","Mining","Success",
    "Agri","Culture","Estates","Plantation","Estate","Station","Expressway","Hospital","Railways","CTB","SLTB","Electricity","Water","Health","Medicine","Social"
    "Doctor","Nurse","School","Power cuts","Fuel","Protest","Strike","Cost","Inflation","Crisis","Corruption","Rights","Education","Politics","University","Rag",
    "Immigration","Visa","Hoppers","Kottu","Rice","Lamprais","Pol","Sambol","Kiribath","Ambul thiyal","Parippu","Roti","Watalappan","Short","Eats","Failure","Student",
    "Achcharu","Concert","Road","Site","Accident","Floods","Barawa","Dengue","Elephant","Leopard","Bear","Whale","Dolphin","Turtle","Birds","Monkey","Challenge",
    "Athletics","Rugby","Volleyball","Football","Soccer","Badminton","Tennis","Swimming","Cycling","Marathon","Hockey","Chess","Cricket","Government","Improve",
    "Movie","Internet","Software","Startup","Startup","Innovation","Research","Development","Science","Technology","App","Website","Platform","Artificial","Budget",
    "Service","Product","Launch","Event","Festival","Conference","Meetup","Workshop","Seminar","Hackathon","Competition","Award","Recognition","Achievement",
    "ලෝකය", "ආරෝග්‍ය", "නිදහස", "ඇතුලත්", "මිතුරන්", "අසරණ", "සඳ","තක්සේරු", "වෙනස්", "අවකාශ", "ආසනය", "ඔබ", "මතක","මැතිවරණ","ප්‍රතිඵල","ශ්‍රී","වාහන","බදු", 
    "සැරසිලි", "චරිත", "විශ්ව", "යුග", "සමාජය", "නව", "අභියෝග", "විවිධ", "රස", "සංස්කෘතිය", "ඇගයීම", "සමෘද්ධි", "නවක", "ජය","අරමුණ", "සංගීතය", "අනුවාදය","ප්‍රවෘත්ති",
    "සහාය", "කෘතීය", "අත්දැකීම්", "කියවීම", "අරමුදල්", "කලා","හොඳ", "බාල", "හිරු", "මනෝභාවය", "විපරිචාරික", "බැලීම", "අවිස්සාවේ", "විද්‍යාව", "ඉතා", "සටහන","කාල", "කෝපි", 
    "අමන", "දැනුම", "ලක්ෂණ", "ඉලක්ක", "ස්ථානය", "විලාසිතාව", "කර්මය", "ජනප්‍රිය","නවීන", "පූජා", "සවිය", "කතාව", "මිතුර", "වටපිටාව", "ජනයා", "ශක්තිය", "මුල්", "අලුත්","සජීවී", 
    "අක්සේපට්", "අපූරු", "ඔපේරාව", "මුලු", "එක්සත්", "ආගම්", "ඉස්ලාමය", "ලෝකය", "අද","දෙවන", "චිත්‍රපට", "අංශ", "මනෝභාව", "බර", "මානව", "දෛනික", "නිර්මාණ", "වෛද්‍ය", 
    "ලක්‍ෂ්‍ය", "සියළු", "අංග", "ආරක්ෂක", "ජාතිය", "බොහෝ", "තොරතුරු", "හඳ", "හරිත","ජාවාරම්කරු","දිවයින","සන්නාමය","සම්මාන","විශේෂාංග","ක්‍රීඩා","චෝදනා","විමර්ශන","ජනාධිපති", 
    "அருட்சக்தி", "இனிய", "நம்பிக்கை", "சக்தி", "பொது", "வாழ்க்கை", "உயிர்", "அறிவு", "நியாயம்", "அமைதி",
    "தொழில்நுட்பம்", "இனியதை", "சினிமா", "அவரை", "புதுக்கால", "கலை", "விருது", "உதவி", "விரும்ப", "கடமை",
    "சுத்தம்", "படை", "ஆதாரம்", "உணர்வு", "நாடுகள்", "சமூகம்", "ஆன்மிகம்", "கலாச்சாரம்", "தூது", "மொழி",
    "இயக்குநர்", "கல்வி", "முத்து", "செம்மொழி", "புதிய", "இல்லம்", "இளமை", "புத்தகம்", "சுவாசம்", "சமயம்",
    "பதிவேற்றம்", "தனிமை", "வசதி", "நகரம்", "மனிதன்", "தூண்டுதல்", "ஆர்வம்", "மிகவும்", "சமூக", "நன்றி"
]

In [5]:
print(len(keywords), "keywords to search")

423 keywords to search


## Text Posts

In [6]:
limit_per_keyword = 1000  
posts_data = []

for keyword in keywords:
    for submission in tqdm(subreddit.search(keyword, limit=limit_per_keyword)):
        try:
            if submission.title.strip() == "" and submission.selftext.strip() == "":
                continue  
            
            posts_data.append({
                    "id": submission.id,
                    "post_title": submission.title,
                    "type": "text",
                    "body": submission.selftext,
                    "score": submission.score,
                    "url": submission.url,
                    "created_utc": submission.created_utc
            })
            
            time.sleep(0.5)  # respect Reddit API rate limit
        except Exception as e:
            print("Error:", e)
            continue

239it [02:07,  1.87it/s]
240it [02:07,  1.88it/s]
38it [00:20,  1.85it/s]
233it [02:05,  1.86it/s]
132it [01:10,  1.87it/s]
97it [00:51,  1.88it/s]
241it [02:11,  1.83it/s]
160it [01:25,  1.86it/s]
102it [00:54,  1.87it/s]
221it [01:57,  1.88it/s]
72it [00:38,  1.88it/s]
237it [02:05,  1.89it/s]
28it [00:15,  1.84it/s]
19it [00:10,  1.79it/s]
152it [01:21,  1.87it/s]
77it [00:41,  1.86it/s]
60it [00:32,  1.86it/s]
230it [02:02,  1.87it/s]
13it [00:07,  1.81it/s]
24it [00:13,  1.81it/s]
245it [02:10,  1.87it/s]
134it [01:12,  1.86it/s]
40it [00:22,  1.82it/s]
56it [00:30,  1.85it/s]
125it [01:06,  1.87it/s]
40it [00:21,  1.84it/s]
237it [02:05,  1.88it/s]
225it [02:00,  1.87it/s]
237it [02:06,  1.87it/s]
230it [02:02,  1.87it/s]
90it [00:47,  1.90it/s]
8it [00:04,  1.74it/s]
26it [00:13,  1.87it/s]
41it [00:21,  1.87it/s]
82it [00:43,  1.87it/s]
101it [00:53,  1.88it/s]
36it [00:19,  1.85it/s]
68it [00:36,  1.88it/s]
79it [00:42,  1.88it/s]
35it [00:18,  1.85it/s]
183it [01:37,  1.89it/

In [7]:
df_text_posts = pd.DataFrame(posts_data)
df_text_posts.head()

,id,post_title,type,body,score,url,created_utc
0,1o5c3kz,In Sri Lanka for a month and shocked,text,I’ve arrived in Sri Lanka a week ago and had a...,307,https://www.reddit.com/r/srilanka/comments/1o5...,1.760336e+09
1,1o21unn,"Sri Lanka, a major disappointment",text,I had commented on how I felt about Sri Lanka ...,8,https://www.reddit.com/r/srilanka/comments/1o2...,1.760004e+09
2,1o0b0v0,Viral TikTok in Sri Lanka,text,"I watch a lot of TikTok videos, and it’s frust...",361,https://v.redd.it/shn4hmez2otf1,1.759833e+09
3,1owpu40,Feel so ashamed as a Sri Lankan😖,text,Felt so ashamed watching this. People on touri...,385,https://i.redd.it/r032jkou961g1.jpeg,1.763104e+09
4,1o39qaf,What’s happening Sri lanka!,text,Why these people are allowed inside Sri lanka?...,722,https://v.redd.it/z2ph50al0cuf1,1.760123e+09


In [8]:
df_text_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38604 entries, 0 to 38603
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           38604 non-null  object 
 1   post_title   38604 non-null  object 
 2   type         38604 non-null  object 
 3   body         38604 non-null  object 
 4   score        38604 non-null  int64  
 5   url          38604 non-null  object 
 6   created_utc  38604 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 2.1+ MB


In [9]:
if not df_text_posts.empty:
    df_text_posts.to_csv("raw data\srilanka_posts_text.csv", index=False, encoding="utf-8")
    print("Data collection complete. Saved to raw data\srilanka_posts_text.csv")
    posts_data.clear()
else:
    print("\nNo text posts were collected.")

Data collection complete. Saved to raw data\srilanka_posts_text.csv


In [27]:
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
from newspaper import Article
import pandas as pd
import time
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

## Non-Text posts

In [28]:
def fetch_html(url):
    """Manual HTML download"""
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        response.raise_for_status()
        return response.text
    except Exception as e:
        print("[HTML ERROR]")
        return None

In [29]:
def extract_article_text(url):
    """Extract text from articles (manual HTML → newspaper.parse)"""
    html = fetch_html(url)
    if html is None:
        return None

    try:
        article = Article(url="")
        article.set_html(html)
        article.parse()
        return article.text
    except Exception as e:
        print(f"[ARTICLE ERROR] {url}")
        return None

In [30]:
def extract_direct_image_text(img_url):
    """Extract OCR text from direct image URL"""
    try:
        r = requests.get(img_url, timeout=10)
        r.raise_for_status()
        img = Image.open(BytesIO(r.content))
        return pytesseract.image_to_string(img)
    except Exception as e:
        print(f"[IMAGE ERROR] {img_url}")
        return None

In [31]:
def extract_gallery_images(submission):
    """Extract OCR text from all images in a gallery post"""
    try:
        items = submission.gallery_data["items"]
        meta = submission.media_metadata
    except Exception as e:
        print(f"[GALLERY METADATA ERROR] {submission.url}")
        return None

    texts = []

    for item in items:
        try:
            media_id = item['media_id']
            img_url = meta[media_id]['s']['u']

            txt = extract_direct_image_text(img_url)
            if txt is not None:
                texts.append(txt)

            time.sleep(0.2)

        except Exception as e:
            print("[GALLERY ITEM ERROR]")

    return texts if texts else None

In [33]:
VALID_IMAGE_EXT = (".jpg", ".jpeg", ".png", ".webp", ".bmp", ".gif")

for keyword in keywords:
    for submission in subreddit.search(keyword, limit=limit_per_keyword):

        url = submission.url.lower()

        # GALLERY POSTS
        if getattr(submission, "is_gallery", False) or "reddit.com/gallery" in url:
            texts = extract_gallery_images(submission)

            if texts is None:
                print("Skipping gallery: ", url)
                continue

            posts_data.append({
                "id": submission.id,
                "post_title": submission.title,
                "type": "gallery",
                "body": texts,
                "score": submission.score,
                "url": submission.url,
                "created_utc": submission.created_utc
            })

            continue

        # IMAGE POSTS
        if url.endswith(VALID_IMAGE_EXT):
            txt = extract_direct_image_text(url)

            if txt is None:
                print("Skipping image: ", url)
                continue

            posts_data.append({
                "id": submission.id,
                "post_title": submission.title,
                "type": "image",
                "body": txt,
                "score": submission.score,
                "url": submission.url,
                "created_utc": submission.created_utc
            })

            continue

        # ARTICLE LINKS
        article_txt = extract_article_text(url)

        if article_txt is None:
            print("Skipping article: ", url)
            continue

        posts_data.append({
            "id": submission.id,
            "post_title": submission.title,
            "type": "article",
            "body": article_txt,
            "score": submission.score,
            "url": submission.url,
            "created_utc": submission.created_utc
        })


[GALLERY ITEM ERROR]
[GALLERY ITEM ERROR]
Skipping gallery:  https://www.reddit.com/gallery/1mgaoic
[HTML ERROR]
Skipping article:  https://gfycat.com/idolizedgracefulblackandtancoonhound
[HTML ERROR]
Skipping article:  https://www.dailymirror.lk/breaking-news/naked-man-arrested-for-riding-motorcycle-to-kandy-from-colombo/108-303492
[HTML ERROR]
Skipping article:  https://www.reddit.com/r/srilanka/comments/1p4baiw/discreet_cafesplaces_in_colombo/
[HTML ERROR]
Skipping article:  https://www.reddit.com/r/srilanka/comments/1ozdve8/three_wheel_repair_shops_in_colombo/
[HTML ERROR]
Skipping article:  https://www.reddit.com/r/srilanka/comments/1oept8b/private_birthday_surprise_places_in_colombo/
[HTML ERROR]
Skipping article:  https://www.reddit.com/r/srilanka/comments/1komr01/how_to_kill_time_at_night_in_colombo/
[HTML ERROR]
Skipping article:  https://www.reddit.com/r/srilanka/comments/1p33yp0/saree_blouse_stitching_colombo/
[HTML ERROR]
Skipping article:  https://www.reddit.com/r/srilanka

KeyboardInterrupt: 

In [34]:
df_non_text_posts = pd.DataFrame(posts_data)
df_non_text_posts.head()

,id,post_title,type,body,score,url,created_utc
0,1o5c3kz,In Sri Lanka for a month and shocked,article,I’ve arrived in Sri Lanka a week ago and had a...,303,https://www.reddit.com/r/srilanka/comments/1o5...,1.760336e+09
1,1o21unn,"Sri Lanka, a major disappointment",article,I had commented on how I felt about Sri Lanka ...,8,https://www.reddit.com/r/srilanka/comments/1o2...,1.760004e+09
2,1o0b0v0,Viral TikTok in Sri Lanka,article,"I watch a lot of TikTok videos, and it’s frust...",366,https://v.redd.it/shn4hmez2otf1,1.759833e+09
3,1owpu40,Feel so ashamed as a Sri Lankan😖,image,,392,https://i.redd.it/r032jkou961g1.jpeg,1.763104e+09
4,1o39qaf,What’s happening Sri lanka!,article,Create your account and connect with a world o...,722,https://v.redd.it/z2ph50al0cuf1,1.760123e+09


In [35]:
df_non_text_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18953 entries, 0 to 18952
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           18953 non-null  object 
 1   post_title   18953 non-null  object 
 2   type         18953 non-null  object 
 3   body         18953 non-null  object 
 4   score        18953 non-null  int64  
 5   url          18953 non-null  object 
 6   created_utc  18953 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.0+ MB


In [36]:
if not df_non_text_posts.empty:
    df_non_text_posts.to_csv("raw data\srilanka_posts_non_text.csv", index=False, encoding="utf-8")
    print("Data collection complete. Saved to raw data\srilanka_posts_non_text.csv")
    posts_data.clear()
else:
    print("\nNo text posts were collected.")

Data collection complete. Saved to raw data\srilanka_posts_non_text.csv


In [37]:
combined_df = pd.concat([df_text_posts, df_non_text_posts], ignore_index=True)

In [38]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57557 entries, 0 to 57556
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           57557 non-null  object 
 1   post_title   57557 non-null  object 
 2   type         57557 non-null  object 
 3   body         57557 non-null  object 
 4   score        57557 non-null  int64  
 5   url          57557 non-null  object 
 6   created_utc  57557 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 3.1+ MB


In [39]:
if not combined_df.empty:
    combined_df.to_csv("raw data\srilanka_all_text.csv", index=False, encoding="utf-8")
    print("Saved to raw data\srilanka_all_text.csv")
else:
    print("\nNo data.")

Saved to raw data\srilanka_all_text.csv
